In [ ]:
node_shape: str = "ellipse"
filter_resources_related_to: str = ""
exclude_incoming_relationships: str = "false"

In [ ]:
import graphviz
import re
from code_data_science import data_table as dt
import code_data_science.palette as palette
import html

df = dt.read_csv("../samples/cobol_relationships.csv")

In [ ]:
graphviz.set_jupyter_format("svg")
dot = graphviz.Digraph(
    "cobol-relationships", comment="COBOL relationships", engine="sfdp"
)

# clean dependent field remove all content up to first slash
df["dependent"] = df["dependent"].apply(lambda x: re.sub(r"^.*\/", "", x))


def apply_filter(node, node_map, seen):
    if node in seen or node not in node_map:
        return

    seen.add(node)
    to = node_map[node]
    for type in to:
        for next in node_map[node][type]:
            apply_filter(next, node_map, seen)


if filter_resources_related_to:
    node_map = {}
    for i, z in enumerate(
        zip(
            df["dependent"], df["dependentType"], df["dependency"], df["dependencyType"]
        )
    ):
        dependent = str(z[0])
        dependentType = str(z[1])
        dependency = str(z[2])
        dependencyType = str(z[3])

        if not bool(node_map):
            node_map = {}

        if dependent not in node_map:
            node_map[dependent] = {}

        if dependentType not in node_map[dependent]:
            node_map[dependent][dependentType] = {}

        if dependency not in node_map[dependent][dependentType]:
            node_map[dependent][dependentType][dependency] = {}

        if dependencyType not in node_map[dependent][dependentType][dependency]:
            node_map[dependent][dependentType][dependency][dependencyType] = {i}

    seen = set()
    apply_filter(filter_resources_related_to, node_map, seen)

    df = df[
        (df["dependent"].isin(seen))
        if exclude_incoming_relationships == "true"
        else (df["dependent"].isin(seen))
        | (df["dependency"].str.contains(filter_resources_related_to))
    ]
    dot.graph_attr["beautify"] = "true"


def style_node(is_missing, dependency_type):
    if is_missing == "true":
        return palette.__moderneColorMap["red"][200]
    if dependency_type == "COBOL":
        return palette.__moderneColorMap["blue"][400]
    if dependency_type == "COPYBOOK":
        return palette.__moderneColorMap["blue"][200]
    if dependency_type == "LINKEDIT":
        return palette.__moderneColorMap["yellow"][200]
    if dependency_type == "BINDPACKAGE":
        return palette.__moderneColorMap["green"][200]
    if dependency_type == "BINDPLAN":
        return palette.__moderneColorMap["green"][400]
    if dependency_type == "SQL_TABLE":
        return palette.__moderneColorMap["indigo"][100]
    if dependency_type == "SQL_CURSOR":
        return palette.__moderneColorMap["indigo"][300]
    return "white"


def add_annotation(value, docType):
    return f"<<b>{html.escape(value)}</b><br/><i>({html.escape(docType)})</i>>"


def map_relationship(row):
    if is_node(row):
        dot.node(
            make_node(row["dependent"], row["dependentType"]),
            shape="Mdiamond"
            if filter_resources_related_to == row["dependent"]
            else node_shape,
            label=add_annotation(row["dependent"], row["dependentType"]),
            style="filled",
            fillcolor=style_node(row["dependencyMissing"], row["dependentType"]),
        )
        dot.node(
            make_node(row["dependency"], row["dependencyType"]),
            shape="Mdiamond"
            if filter_resources_related_to == row["dependency"]
            else node_shape,
            label=add_annotation(row["dependency"], row["dependencyType"]),
            style="filled",
            fillcolor=style_node(row["dependencyMissing"], row["dependencyType"]),
        )
        dot.edge(
            make_node(row["dependent"], row["dependentType"]),
            make_node(row["dependency"], row["dependencyType"]),
            label=make_label(
                row["action"], row["actionMetadata"] if "actionMetadata" in df else None
            ),
        )


# Prevent multiple redundant relationships from being created when multiple COBOL sources access the same copybook
copy_chain_pairs = {}


def is_node(row):
    if (
        isinstance(row["dependent"], str)
        and isinstance(row["dependentType"], str)
        and row["dependentType"] == "COPYBOOK"
        and isinstance(row["dependency"], str)
        and isinstance(row["dependencyType"], str)
        and row["dependencyType"] == "COPYBOOK"
    ):
        dependent = row["dependent"]
        dependency = row["dependency"]
        if dependent not in copy_chain_pairs:
            copy_chain_pairs[dependent] = {}
            copy_chain_pairs[dependent][dependency] = dependency
            return True
        elif dependency not in copy_chain_pairs[dependent]:
            copy_chain_pairs[dependent][dependency] = dependency
            return True
    else:
        return True


def make_node(resource, type):
    if isinstance(resource, str) and isinstance(type, str):
        return resource + " " + type
    else:
        return resource


def make_label(action, action_metadata):
    if isinstance(action_metadata, str):
        return action + " " + f"({action_metadata})"
    else:
        return action


df.apply(map_relationship, axis=1)

dot.graph_attr = {
    "overlap": "prism",
    "normalize": "true",
    "overlap_scaling": "30",
    "size": "1250,auto",
    "nodesep": "1",
}

dot